# Experiment
> Can we get better by training on our assumptions?

In [ ]:
# default_exp old___experiment

In [ ]:
# hide
import blackhc.project.script

Import modules and functions were are going to use.

In [ ]:
# exports

import dataclasses
import traceback
from dataclasses import dataclass
from typing import Type, Union

import torch
import torch.utils.data
from blackhc.project import is_run_from_ipython
from blackhc.project.experiment import embedded_experiments
from torch import nn
from torch.utils.data import Dataset

import batchbald_redux.acquisition_functions as acquisition_functions
from batchbald_redux.acquisition_functions import (
    CandidateBatchComputer,
    EvalModelBatchComputer,
)
from batchbald_redux.active_learning import ActiveLearningData
from batchbald_redux.black_box_model_training import evaluate
from batchbald_redux.dataset_challenges import (
    create_repeated_MNIST_dataset,
    get_base_dataset_index,
    get_target,
)
from batchbald_redux.di import DependencyInjection
from batchbald_redux.models import MnistModelTrainer
from batchbald_redux.experiment_data import ExperimentData, ExperimentDataConfig

In [ ]:
# exports

from batchbald_redux.train_eval_model import (
    TrainEvalModel,
    TrainSelfDistillationEvalModel,
)
from batchbald_redux.trained_model import ModelTrainer

In [ ]:
# exports


@dataclass
class Experiment:
    seed: int = 1337
    acquisition_size: int = 5
    max_training_set: int = 300
    num_pool_samples: int = 20
    num_validation_samples: int = 20
    num_training_samples: int = 1
    num_patience_epochs: int = 5 * 4
    max_training_epochs: int = 30 * 4
    training_batch_size: int = 64
    device: str = "cuda"
    validation_set_size: int = 2048
    initial_set_size: int = 20
    min_samples_per_epoch: int = 1024
    repeated_mnist_repetitions: int = 1
    add_dataset_noise: bool = False
    acquisition_function: Union[
        Type[CandidateBatchComputer], Type[EvalModelBatchComputer]
    ] = acquisition_functions.BALD
    train_eval_model_factory: Type[TrainEvalModel] = TrainSelfDistillationEvalModel
    model_trainer_factory: Type[ModelTrainer] = MnistModelTrainer
    acquisition_function_args: dict = None
    temperature: float = 0.0

    def load_dataset(self) -> (ActiveLearningData, Dataset, Dataset):
        experiment_data_config = ExperimentDataConfig(id_dataset_name="MNIST",
                                                      id_repetitions=self.repeated_mnist_repetitions,
                                                      initial_training_set_size=self.initial_set_size,
                                                      evaluation_set_size=0,
                                                      validation_split_random_state=0,
                                                      validation_set_size=self.validation_set_size,
                                                      add_dataset_noise=self.add_dataset_noise, ood_dataset_config=None,
                                                      device=self.device)
        experiment_data = experiment_data_config.load()

        return experiment_data.active_learning, experiment_data.validation_dataset, experiment_data.test_dataset

    # Simple Dependency Injection
    def create_acquisition_function(self):
        di = DependencyInjection(vars(self))
        return di.create_dataclass_type(self.acquisition_function)

    def create_train_eval_model(self, runtime_config) -> TrainEvalModel:
        config = {**vars(self), **runtime_config}
        di = DependencyInjection(config, [])
        return di.create_dataclass_type(self.train_eval_model_factory)

    def create_model_trainer(self) -> ModelTrainer:
        di = DependencyInjection(vars(self))
        return di.create_dataclass_type(self.model_trainer_factory)

    def run(self, store):
        torch.manual_seed(self.seed)

        # Active Learning setup
        active_learning_data, validation_dataset, test_dataset = self.load_dataset()
        store["dataset_info"] = dict(training=repr(active_learning_data.base_dataset), test=repr(test_dataset))

        model_trainer = self.create_model_trainer()

        train_loader = model_trainer.get_train_dataloader(active_learning_data.training_dataset)
        pool_loader = model_trainer.get_evaluation_dataloader(active_learning_data.pool_dataset)
        validation_loader = model_trainer.get_evaluation_dataloader(validation_dataset)
        test_loader = model_trainer.get_evaluation_dataloader(test_dataset)

        store["active_learning_steps"] = []
        active_learning_steps = store["active_learning_steps"]

        acquisition_function = self.create_acquisition_function()

        # Active Training Loop
        while True:
            training_set_size = len(active_learning_data.training_dataset)
            print(f"Training set size {training_set_size}:")

            # iteration_log = dict(training={}, pool_training={}, evaluation_metrics=None, acquisition=None)
            active_learning_steps.append({})
            iteration_log = active_learning_steps[-1]

            iteration_log["training"] = {}
            trained_model = model_trainer.get_trained(train_loader=train_loader, train_augmentations=None,
                                                      validation_loader=validation_loader,
                                                      log=iteration_log["training"])

            evaluation_metrics = evaluate(model=trained_model, num_samples=self.num_validation_samples,
                                          loader=test_loader, device=self.device, storage_device="cpu")

            iteration_log["evaluation_metrics"] = evaluation_metrics
            print(f"Perf after training {evaluation_metrics}")

            if training_set_size >= self.max_training_set:
                print("Done.")
                break

            if isinstance(acquisition_function, CandidateBatchComputer):
                candidate_batch = acquisition_function.compute_candidate_batch(trained_model, pool_loader, self.device)
            elif isinstance(acquisition_function, EvalModelBatchComputer):
                train_eval_model = self.create_train_eval_model(
                    dict(
                        model_trainer=model_trainer,
                        training_dataset=active_learning_data.training_dataset,
                        eval_dataset=active_learning_data.pool_dataset,
                        validation_loader=validation_loader,
                        trained_model=trained_model,
                    )
                )

                iteration_log["eval_training"] = {}
                trained_eval_model = train_eval_model(training_log=iteration_log["eval_training"], device=self.device)

                candidate_batch = acquisition_function.compute_candidate_batch(
                    trained_model, trained_eval_model, pool_loader, device=self.device
                )
            else:
                raise ValueError(f"Unknown acquisition function {acquisition_function}!")

            candidate_global_indices = [
                get_base_dataset_index(active_learning_data.pool_dataset, index).index
                for index in candidate_batch.indices
            ]
            candidate_labels = [
                get_target(active_learning_data.base_dataset, index).item() for index in candidate_global_indices
            ]

            iteration_log["acquisition"] = dict(
                indices=candidate_global_indices, labels=candidate_labels, scores=candidate_batch.scores
            )

            active_learning_data.acquire(candidate_batch.indices)

            ls = ", ".join(f"{label} ({score:.4})" for label, score in zip(candidate_labels, candidate_batch.scores))
            print(f"Acquiring (label, score)s: {ls}")

In [ ]:
# experiment

experiment = Experiment(
    seed=1120,
    max_training_epochs=2,
    max_training_set=130,
    acquisition_function=acquisition_functions.BatchEvalBALD,
    acquisition_size=2,
    num_pool_samples=20,
    temperature=8,
    device="cuda",
)

results = {}
experiment.run(results)

Creating: MnistModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	num_patience_epochs=20,
	max_training_epochs=2,
	min_samples_per_epoch=1024
)
Creating: BatchEvalBALD(
	acquisition_size=2,
	num_pool_samples=20
)
Training set size 20:


 50%|#####     | 1/2 [00:00<?, ?it/s]

[1/16]   6%|6          [00:00<?]

[1/16]   6%|6          [00:00<?]

Epoch metrics: {'accuracy': 0.59033203125, 'crossentropy': 1.8616515100002289}


...

[1/16]   6%|6          [00:00<?]

[1/16]   6%|6          [00:00<?]

Epoch metrics: {'accuracy': 0.5341796875, 'crossentropy': 1.5239248871803284}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.5341796875)
RestoringEarlyStopping: Restoring optimizer.


get_predictions_labels:   0%|          | 0/1158600 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
results

{'initial_training_set_indices': [38043,
  40091,
  17418,
  2094,
  39879,
  3133,
  5011,
  40683,
  54379,
  24287,
  9849,
  59305,
  39508,
  39356,
  8758,
  52579,
  13655,
  7636,
  21562,
  41329],
 'dataset_info': {'training': "'FastMNIST (Train)'",
  'test': "'FastMNIST (Test)'"},
 'active_learning_steps': [{'training': {'epochs': [{'accuracy': 0.59033203125,
      'crossentropy': 1.8616515100002289},
     {'accuracy': 0.6083984375, 'crossentropy': 1.8942454904317856}],
    'best_epoch': 2},
   'evaluation_metrics': {'accuracy': 0.618, 'crossentropy': tensor(1.2012)},
   'eval_training': {'epochs': [{'accuracy': 0.42822265625,
      'crossentropy': 1.880868785083294},
     {'accuracy': 0.60302734375, 'crossentropy': 1.4887847229838371}],
    'best_epoch': 2},
   'acquisition': {'indices': [54709, 24497],
    'labels': [2, 9],
    'scores': [0.7566147013141693, 1.2705203278051234]}},
  {'training': {'epochs': [{'accuracy': 0.3779296875,
      'crossentropy': 2.018156968057155

In [ ]:
# experiment

experiment = Experiment(
    max_training_epochs=1, max_training_set=25, acquisition_function=AcquisitionFunction.randombaldical
)

results = {}
experiment.run(results)

results

Training set size 20:


100%|##########| 1/1 [00:00<?, ?it/s]

[1/384]   0%|           [00:00<?]

[1/64]   2%|1          [00:00<?]

RestoringEarlyStopping: Restoring best parameters. (Score: -6.529030114412308)
RestoringEarlyStopping: Restoring optimizer.


...

[1/64]   2%|1          [00:00<?]

RestoringEarlyStopping: Restoring best parameters. (Score: -4.6851686127483845)
RestoringEarlyStopping: Restoring optimizer.


[1/157]   1%|           [00:00<?]

Perf after training {'accuracy': 0.6256, 'crossentropy': 4.484497045135498}
Done.


{'initial_training_set_indices': [38043,
  40091,
  17418,
  2094,
  39879,
  3133,
  5011,
  40683,
  54379,
  24287,
  9849,
  59305,
  39508,
  39356,
  8758,
  52579,
  13655,
  7636,
  21562,
  41329],
 'active_learning_steps': [{'training': {'epochs': [{'accuracy': 0.538818359375,
      'crossentropy': 6.529030114412308}],
    'best_epoch': 1},
   'evalution_metrics': {'accuracy': 0.5367,
    'crossentropy': 6.438035237884521},
   'pool_training': {'epochs': [{'accuracy': 0.531005859375,
      'crossentropy': 5.1637596152722836}],
    'best_epoch': 1},
   'acquisition': {'indices': [63338, 10856, 63452, 81864, 109287],
    'labels': [8, 8, 3, 3, 3],
    'scores': [0.8710822958846325,
     0.8687216999221631,
     0.8759664372823723,
     0.8464646732511746,
     0.8810812784952251]}},
  {'training': {'epochs': [{'accuracy': 0.62255859375,
      'crossentropy': 4.6851686127483845}],
    'best_epoch': 1},
   'evalution_metrics': {'accuracy': 0.6256,
    'crossentropy': 4.4844970451

In [ ]:
if False:
    configs = (
        [
            Experiment(
                seed=seed + 120,
                acquisition_function=acquisition_function,
                acquisition_size=acquisition_size,
                num_pool_samples=100,
            )
            for seed in range(5)
            for acquisition_size in [5, 10]
            for acquisition_function in [AcquisitionFunction.batchbald, AcquisitionFunction.batchbaldical]
        ]
        + [
            Experiment(
                seed=seed + 120,
                acquisition_function=acquisition_function,
                acquisition_size=acquisition_size,
                num_pool_samples=max(20, acquisition_size),
            )
            for seed in range(5)
            for acquisition_size in [5, 10, 20, 50]
            for acquisition_function in [
                AcquisitionFunction.bald,
                AcquisitionFunction.thompsonbald,
                AcquisitionFunction.randombald,
            ]
        ]
        + [
            Experiment(
                seed=seed + 120,
                acquisition_function=AcquisitionFunction.random,
                acquisition_size=5,
                num_pool_samples=20,
            )
            for seed in range(40)
            for acquisition_size in [5]
        ]
    )
if False:
    configs = [
        Experiment(
            seed=seed + 240,
            acquisition_function=acquisition_function,
            acquisition_size=acquisition_size,
            num_pool_samples=max(20, acquisition_size),
        )
        for seed in range(5)
        for acquisition_size in [5, 10, 20, 50]
        for acquisition_function in [
            AcquisitionFunction.baldical,
            AcquisitionFunction.randombaldical,
        ]
    ]
if False:
    configs = [
        Experiment(
            seed=seed + 340,
            acquisition_function=acquisition_function,
            acquisition_size=acquisition_size,
            num_pool_samples=20,
            temperature=temperature,
        )
        for seed in range(5)
        for acquisition_size in [5, 10, 20, 50]
        for acquisition_function in [
            AcquisitionFunction.temperedbald,
        ]
        for temperature in [13, 15, 18]
    ]
if False:
    configs = [
        Experiment(
            seed=seed + 340,
            acquisition_function=acquisition_function,
            acquisition_size=acquisition_size,
            num_pool_samples=20,
            temperature=temperature,
        )
        for seed in range(5)
        for acquisition_size in [5, 10, 20, 50]
        for acquisition_function in [
            AcquisitionFunction.temperedbald,
        ]
        for temperature in [8, 10]
    ] + [
        Experiment(
            seed=seed + 340,
            acquisition_function=acquisition_function,
            acquisition_size=acquisition_size,
            num_pool_samples=20,
            temperature=temperature,
        )
        for seed in range(5)
        for acquisition_size in [5, 10, 20, 50]
        for acquisition_function in [
            AcquisitionFunction.temperedbaldical,
        ]
        for temperature in [8, 10, 13]
    ]
if False:
    configs = [
        Experiment(
            seed=seed + 500,
            acquisition_function=acquisition_function,
            acquisition_size=acquisition_size,
            num_pool_samples=20,
            temperature=temperature,
        )
        for seed in range(5)
        for acquisition_size in [5, 10, 20, 50]
        for acquisition_function in [
            AcquisitionFunction.temperedical,
        ]
        for temperature in [5, 8, 11]
    ] + [
        Experiment(
            seed=seed + 600,
            acquisition_function=acquisition_function,
            acquisition_size=acquisition_size,
            num_pool_samples=20,
        )
        for seed in range(5)
        for acquisition_size in [5, 10, 20, 50]
        for acquisition_function in [
            AcquisitionFunction.ical,
        ]
    ]
if False:
    configs = (
        [
            Experiment(
                seed=seed + 1000,
                acquisition_function=acquisition_function,
                acquisition_size=acquisition_size,
                num_pool_samples=num_pool_samples,
                temperature=temperature,
            )
            for seed in range(5)
            for acquisition_size in [5, 10, 20, 50]
            for num_pool_samples in [20, 100]
            for acquisition_function in [
                AcquisitionFunction.temperedical,
                AcquisitionFunction.temperedbald,
                AcquisitionFunction.temperedbaldical,
            ]
            for temperature in [2, 5, 8]
        ]
        + [
            Experiment(
                seed=seed + 2000,
                acquisition_function=acquisition_function,
                acquisition_size=acquisition_size,
                num_pool_samples=num_pool_samples,
            )
            for seed in range(5)
            for acquisition_size in [5, 10, 20, 50]
            for num_pool_samples in [20, 100]
            for acquisition_function in [
                AcquisitionFunction.ical,
                AcquisitionFunction.bald,
                AcquisitionFunction.baldical,
                AcquisitionFunction.randombald,
            ]
        ]
        + [
            Experiment(
                seed=seed + 2000,
                acquisition_function=acquisition_function,
                acquisition_size=acquisition_size,
                num_pool_samples=max(num_pool_samples, acquisition_size),
            )
            for seed in range(5)
            for acquisition_size in [5, 10, 20, 50]
            for num_pool_samples in [20, 100]
            for acquisition_function in [
                AcquisitionFunction.thompsonbald,
            ]
        ]
        + [
            Experiment(
                seed=seed + 3000,
                acquisition_function=acquisition_function,
                acquisition_size=acquisition_size,
                num_pool_samples=100,
            )
            for seed in range(5)
            for acquisition_size in [5]
            for acquisition_function in [
                AcquisitionFunction.batchbaldical,
                AcquisitionFunction.batchbald,
            ]
        ]
    )
if False:
    configs = [
    Experiment(
        seed=seed,
        acquisition_function=acquisition_functions.BALD,
        acquisition_size=acquisition_size,
        num_pool_samples=num_pool_samples,
    )
    for seed in range(5)
    for acquisition_size in [5, 10, 20, 50]
    for num_pool_samples in [10, 20, 50, 100]
] + [
    Experiment(
        seed=seed,
        acquisition_function=acquisition_functions.Random,
        acquisition_size=5,
        num_pool_samples=20,
    )
    for seed in range(20)
]


In [ ]:
# exports

configs = [
    Experiment(
        seed=seed + 315,
        acquisition_function=acquisition_function,
        acquisition_size=acquisition_size,
        num_pool_samples=num_pool_samples,
        repeated_mnist_repetitions=repeated_mnist_repetitions,
        add_dataset_noise=repeated_mnist_repetitions > 1,
        temperature=temperature,
        max_training_set=150,
    )
    for seed in range(5)
    for acquisition_function in [
        acquisition_functions.SoftmaxBALD,
    ]
    for temperature in [1 / 32, 1 / 64, 1 / 128, 1 / 256]
    for acquisition_size in [5]
    for num_pool_samples in [100]
    for repeated_mnist_repetitions in [2]
]

if not is_run_from_ipython() and __name__ == "__main__":
    for job_id, store in embedded_experiments(__file__, len(configs)):
        config = configs[job_id]
        config.seed += job_id
        print(config)
        store["config"] = dataclasses.asdict(config)
        store["log"] = {}

        try:
            config.run(store=store)
        except Exception:
            store["exception"] = traceback.format_exc()
            raise

In [ ]:
len(configs)

90

In [ ]:
# slow
import prettyprinter

prettyprinter.install_extras(include={"dataclasses"})

prettyprinter.pprint(configs)

[
    Experiment(
        seed=115,
        max_training_set=150,
        num_pool_samples=100,
        repeated_mnist_repetitions=2,
        add_dataset_noise=True,
        # class
        acquisition_function=batchbald_redux.acquisition_functions.SoftmaxBALD,
        temperature=0.0625
    ),
    Experiment(
        seed=115,
        max_training_set=150,
        num_pool_samples=100,
        repeated_mnist_repetitions=2,
        add_dataset_noise=True,
        # class
        acquisition_function=batchbald_redux.acquisition_functions.SoftmaxBALD,
        temperature=0.125
    ),
    Experiment(
        seed=115,
        max_training_set=150,
        num_pool_samples=100,
        repeated_mnist_repetitions=2,
        add_dataset_noise=True,
        # class
        acquisition_function=batchbald_redux.acquisition_functions.SoftmaxBALD,
        temperature=0.25
    ),
    Experiment(
        seed=115,
        max_training_set=150,
        num_pool_samples=100,
        repeated_mnist_re